In [4]:
import time
import os
import numpy as np
from lib.utilities import Dataset
from lib.file.zda_writer import ZDA_Writer

In [5]:
# written for Anna's old-rig issue of missing RLI frames
# takes early pre-stim frames and averages them into an RLI frame in .ZDA file
data_dir = "C:/Users/jjudge3/Desktop/Data/Anna's/converted_zda/"
pre_stim_range = [25, 85]  # frames
num_fp_pts = 4

In [6]:
# read in .ZDA file and overwrite only the RLI portion
zda_writer = ZDA_Writer()
for filename in os.listdir(data_dir):
    if filename.endswith(".zda"):
        filename = data_dir + filename
        print("Working on", filename)
        ds = Dataset(filename)
        
        # fetch some necessary metadata
        meta = ds.get_meta()
        
        ds.clip_data(t_range=pre_stim_range,
                     x_range=[0, meta['raw_width']],
                     y_range=[0, meta['raw_height']])
        frames = ds.get_data()
        
        all_frames = ds.get_unclipped_data()
        fp_array = ds.get_fp_data()
        print(frames.shape)
        
        # calculate RLI frames
        rli_high = np.average(frames, axis=1)[0, :, :].reshape(-1)  # only from first trial
        rli_max = np.max(frames, axis=1)[0, :, :].reshape(-1)  # only from first trial

        # normalize to 3278
        #minn = np.min(rli_high)
        #rli_high = rli_high - minn
        #maxx = np.max(rli_high)
        #rli_high = rli_high / maxx * (3278 / 3)
        
        rli_low = np.zeros(rli_high.shape)
        rli = {'rli_low': rli_low,
               'rli_high': rli_high,
               'rli_max': rli_max }
        
        print(rli_high.shape)
        
        # overwrite back
        zda_writer.overwrite_all_except_meta_header(all_frames, meta, filename, rli, fp_array)
        print("... done")
        

Working on C:/Users/jjudge3/Desktop/Data/Anna's/converted_zda/01_01_01.zda
(10, 60, 80, 80)
(6400,)
wrote 600 points for 80 x 80 x 10 px
... done
